# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [41]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

In [42]:
from huggingface_hub import notebook_login

notebook_login()

In [43]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [44]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [62]:
from smolagents import LiteLLMModel, CodeAgent

model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-lite", api_key=userdata.get(key="GEMINI_API_KEY"),provider='together')

We can start with creating a baseline, simple agent to give us a simple report.

In [63]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [64]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [73]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(provider="serpapi"), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [74]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  batman_locations_search = web_search(query="Batman filming locations")                                           
  print(batman_locations_search)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George's Hall, Liverpool, England, UK. (Exterior ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [The Batman: A Dark and Gritty Reboot of the Iconic 
...](https://www.latlong.net/location/the-batman-a-dark-and-gritty-reboot-of-the-iconic-superhero-locations-427)
Source: Latitude and Longitude Finder

The Batman: A Dark and Gritty Reboot of the Iconic Superhero was filmed in 2 Temple Pl, 230 S LaSelle St, 91 
Wishart St, Anfield Cemetery, Cardington Studios, ...

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqu-sjyyDy1za3Cu7k2nK54RxS5_NgXLfNG7TJ
IPxmQA4qPYfnC)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [The Batman (film)](https://en.wikipedia.org/wiki/The_Batman_(film))
Source: Wikipedia

Filming began in the United Kingdom in January 2020 but was halted by the ... Unlike previous Batman films, The 
Batman focuses on Batman's detective ...

8. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

9. [Locations where The Batman movie starring Robert 
...](https://www.whtimes.co.uk/news/22344457.revealed-locations-batman-movie-starring-robert-pattinson-filmed/)
Date published: Mar 3, 2022
Source: Welwyn Hatfield Times

As well as being filmed on location in Chicago, America, and London, Liverpool and Glasgow in the UK, The Batman 
was also shot on sets built at ...

Out: None

[Step 1: Duration 1.59 seconds| Input tokens: 2,578 | Output tokens: 152]

...

[Step 7: Duration 3.27 seconds| Input tokens: 31,358 | Output tokens: 1,548]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(combined_df.to_string())                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                                           Travel Time (hours)
London, UK                                               9.19
Liverpool, UK                                            9.36
Glasgow, UK                                              9.35
Chicago, USA                                             2.68
New York, USA                                            1.00
Pittsburgh, USA                                          1.74
Lamborghini, Sant'Agata Bolognese, Italy                10.76
Ferrari, Maranello, Italy                               10.73
Porsche, Stuttgart, Germany                             10.24
McLaren, Woking, UK                                      9.14

[Step 8: Duration 0.76 seconds| Input tokens: 38,563 | Output tokens: 1,587]

In [67]:
result

,travel_time_hours
Necropolis Cemetery,8.60
Glasgow,9.35
Scotland,9.31
. ; St. George's Hall,9.36
Liverpool,9.36
England,9.31
Knebworth House,17.37
Knebworth,9.14
Hertfordshire,9.19
Acton Lane,16.14


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [70]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## Facts survey
### 1.1. Facts given in the task
*   The task requires finding Batman filming locations worldwide.
*   The task requires calculating the cargo plane transfer time from each filming location to Gotham (40.7128° N, 
74.0060° W).
*   The task requires finding supercar factories.
*   The task requires finding supercar factories with a cargo plane transfer time to Gotham similar to the Batman 
filming locations.

### 1.2. Facts to look up
*   Batman filming locations: A list of locations where Batman movies were filmed. Sources: web search (multiple 
queries may be needed)
*   Latitude and Longitude for Batman filming locations: For use with calculate_cargo_travel_time. Sources: web 
search (after identifying location)
*   Supercar factories locations: Locations of supercar factories. Sources: web search (multiple queries may be 
needed)
*   Latitude and Longitude for Supercar factories locations: For use with calculate_cargo_travel_time. Sources: web
search (after identifying location)

### 1.3. Facts to derive
*   Cargo plane travel time from each Batman filming location to Gotham: Calculated using 
`calculate_cargo_travel_time`.
*   Cargo plane travel time from each supercar factory to Gotham: Calculated using `calculate_cargo_travel_time`.
*   Similarity in cargo plane travel times: Comparing the travel times from Batman locations and supercar 
factories.

## Plan
1.  **Find Batman filming locations**: Use `web_search` to search for "Batman filming locations".
2.  **Verify Batman filming locations**: Iterate through the results from the initial search, and for each 
potential location found, use `visit_webpage` to visit the source URL and verify the information.
3.  **Gather location coordinates for Batman locations**: For each verified Batman filming location, use 
`web_search` to search for "latitude longitude [location name]".
4.  **Calculate travel times for Batman locations**: For each Batman filming location's coordinates, use 
`calculate_cargo_travel_time` to determine travel time to Gotham (40.7128° N, 74.0060° W).
5.  **Find Supercar factories**: Use `web_search` to search for "supercar factories locations".
6.  **Verify Supercar factory locations**: Iterate through the results from the initial search, and for each 
potential location found, use `visit_webpage` to visit the source URL and verify the information.
7.  **Gather location coordinates for Supercar factories**: For each verified supercar factory location, use 
`web_search` to search for "latitude longitude [factory name]".
8.  **Calculate travel times for Supercar factories**: For each supercar factory's coordinates, use 
`calculate_cargo_travel_time` to determine travel time to Gotham (40.7128° N, 74.0060° W).
9.  **Compare travel times**: Compare the calculated cargo plane travel times for Batman filming locations and 
supercar factories.
10. **Create the DataFrame**: Construct a pandas DataFrame containing Batman filming locations, their coordinates, 
and their travel times to Gotham.
11. **Find Supercar Factories with similar travel times**: Identify supercar factories with travel times close to 
the Batman locations.
12. **Final Answer**: Provide the pandas DataFrame and the list of supercar factories that are near the Batman 
locations in terms of travel time using `final_answer`.
\n
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  import re                                                                                                        
                                                                                                                   
  # --- Batman Filming Locations ---                                                                               
  batman_locations = []                                                                                            
  batman_coords = []                                                                                               
  batman_travel_times = []                                                                                         
                                                                                                                   
  query = "Batman filming locations"                                                                               
  search_results = web_search(query)                                                                               
  print(search_results)                                                                                            
                                                                                                                   
  # Extract URLs and Filming Locations                                                                             
  location_matches = re.findall(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+[^'\"]*", search_results)                 
  print(f"Found {len(location_matches)} potential Batman filming locations.")                                      
                                                                                                                   
  for url_index, url in enumerate(location_matches[:3]):                                                           
      try:                                                                                                         
          page_content = visit_webpage(url)                                                                        
          print(f"Visiting webpage {url_index+1}: {url}")                                                          
          # Basic check for "Batman" or related keywords to confirm the information                                
          if "batman" in page_content.lower() or "movie locations" in page_content.lower():                        
              # Extract locations from page_content                                                                
              locations = re.findall(r"([A-Za-z\s,]+(?:, [A-Za-z]{2,3})?)", page_content) # improved regex         
              print(f"Found {len(locations)} locations on page.")                                                  
                                                                                                                   
              for location in set(locations):                                                                      
                  if len(location.strip()) > 3: # to skip short strings                                            
                      print(f"Attempting to retrieve coordinates for {location}")                                  
                      coord_query = f"latitude longitude {location}"                                               
                      coord_results = web_search(coord_query)                                                      
                      coord_match = re.search(r"(\-?\d+\.\d+).*?(\-?\d+\.\d+)", coord_results)                     
                      if coord_match:                                                                              
  [38;2;248;248;24

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George's Hall, Liverpool, England, UK. (Exterior ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [The Batman: A Dark and Gritty Reboot of the Iconic 
...](https://www.latlong.net/location/the-batman-a-dark-and-gritty-reboot-of-the-iconic-superhero-locations-427)
Source: Latitude and Longitude Finder

The Batman: A Dark and Gritty Reboot of the Iconic Superhero was filmed in 2 Temple Pl, 230 S LaSelle St, 91 
Wishart St, Anfield Cemetery, Cardington Studios, ...

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqu-sjyyDy1za3Cu7k2nK54RxS5_NgXLfNG7TJ
IPxmQA4qPYfnC)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [The Batman (film)](https://en.wikipedia.org/wiki/The_Batman_(film))
Source: Wikipedia

Filming began in the United Kingdom in January 2020 but was halted by the ... Unlike previous Batman films, The 
Batman focuses on Batman's detective ...

8. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

9. [Locations where The Batman movie starring Robert 
...](https://www.whtimes.co.uk/news/22344457.revealed-locations-batman-movie-starring-robert-pattinson-filmed/)
Date published: Mar 3, 2022
Source: Welwyn Hatfield Times

As well as being filmed on location in Chicago, America, and London, Liverpool and Glasgow in the UK, The Batman 
was also shot on sets built at ...
Found 3 potential Batman filming locations.
Visiting webpage 1: https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George
Found 29 locations on page.
Attempting to retrieve coordinates for locations
Could not find coordinates for locations
Attempting to retrieve coordinates for Batman
Coordinates found for Batman: (37.714108, 41.413166)
Attempting to retrieve coordinates for  https
Could not find coordinates for  https
Attempting to retrieve coordinates for   for url
Coordinates found for   for url: (41.900196, 12.488256)
Attempting to retrieve coordinates for Cemetery,
Coordinates found for Cemetery,: (31.7, 31.4)
Attempting to retrieve coordinates for ASource
Could not find coordinates for ASource
Attempting to retrieve coordinates for Glasgow,
Co

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  import re                                                                                                        
                                                                                                                   
  # --- Batman Filming Locations ---                                                                               
  batman_locations = ["Glasgow, Scotland", "Liverpool, England", "London, England", "Chicago, USA"]                
  batman_coords = [(55.8642, -4.2518), (53.4084, -2.9978), (51.5074, -0.1278), (41.8781, -87.6298)]                
  batman_travel_times = []                                                                                         
                                                                                                                   
  # Calculate Travel Times for Batman Locations                                                                    
  gotham_coords = (40.7128, -74.0060) # Gotham coords                                                              
  for coords in batman_coords:                                                                                     
      try:                                                                                                         
          travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)        
          batman_travel_times.append(travel_time)                                                                  
      except Exception as e:                                                                                       
          print(f"Error calculating travel time for {coords}: {e}")                                                
          batman_travel_times.append(None) # in case of an error                                                   
                                                                                                                   
  # --- Supercar Factories ---                                                                                     
  supercar_factories = ["Ferrari, Maranello, Italy", "Lamborghini, Bologna, Italy", "McLaren, Woking, UK"]         
  supercar_coords = [(44.5311, 10.8680), (44.5389, 11.5921), (51.3482, -0.5567)]                                   
  supercar_travel_times = []                                                                                       
                                                                                                                   
  # Calculate Travel Times for Supercar Factories                                                                  
  for coords in supercar_coords:                                                                                   
      try:                                                                                                         
          travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)        
          supercar_travel_times.append(travel_time)                                                                
      except Exception as e:                                                                                       
          print(f"Error calculating travel time for {coords}: {e}")                                                
          supercar_travel_times.append(None)                                                                       
                                                                                                                   
  # Create DataFrame                                                                                               
  batman_df = pd.DataFrame({                             

Execution logs:
             Location  Latitude  Longitude  Travel Time to Gotham (hours)
0   Glasgow, Scotland   55.8642    -4.2518                           8.60
1  Liverpool, England   53.4084    -2.9978                           8.81
2     London, England   51.5074    -0.1278                           9.17
3        Chicago, USA   41.8781   -87.6298                           2.68
Supercar Factories with similar travel times: ['Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 
'McLaren, Woking, UK', 'Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK', 'Ferrari,
Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK']

Out - Final answer: Batman Filming Locations:
             Location  Latitude  Longitude  Travel Time to Gotham (hours)
0   Glasgow, Scotland   55.8642    -4.2518                           8.60
1  Liverpool, England   53.4084    -2.9978                           8.81
2     London, England   51.5074    -0.1278                           9.17
3        Chicago, USA   41.8781   -87.6298                           2.68

Supercar Factories with similar travel times: ['Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 
'McLaren, Woking, UK', 'Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK', 'Ferrari,
Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK']

[Step 2: Duration 5.00 seconds| Input tokens: 15,169 | Output tokens: 2,399]

Batman Filming Locations:
             Location  Latitude  Longitude  Travel Time to Gotham (hours)
0   Glasgow, Scotland   55.8642    -4.2518                           8.60
1  Liverpool, England   53.4084    -2.9978                           8.81
2     London, England   51.5074    -0.1278                           9.17
3        Chicago, USA   41.8781   -87.6298                           2.68

Supercar Factories with similar travel times: ['Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK', 'Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK', 'Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK']


In [71]:
detailed_report

"Batman Filming Locations:\n             Location  Latitude  Longitude  Travel Time to Gotham (hours)\n0   Glasgow, Scotland   55.8642    -4.2518                           8.60\n1  Liverpool, England   53.4084    -2.9978                           8.81\n2     London, England   51.5074    -0.1278                           9.17\n3        Chicago, USA   41.8781   -87.6298                           2.68\n\nSupercar Factories with similar travel times: ['Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK', 'Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK', 'Ferrari, Maranello, Italy', 'Lamborghini, Bologna, Italy', 'McLaren, Woking, UK']"

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [75]:
model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-lite", api_key=userdata.get(key="GEMINI_API_KEY"), provider="together", max_tokens=8096)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serpapi"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [76]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [87]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [88]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gemini/gemini-2.0-flash-lite
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.        

In [89]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

[Step 1: Duration 0.19 seconds]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-67ed073d-40380e9f190d96905297ba1e;af4fc6ab-2abf-4fab-add8-2849c0eaaafd)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [86]:
manager_agent.python_executor.state["fig"]

KeyError: 'fig'

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)